In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sqlite3
%matplotlib inline
from IPython.core.display import HTML
pd.options.display.max_columns = 50
import seaborn as sb
import datetime
HTML("<style>.container { width:100% !important; }</style>")

In [41]:
sql_conn = sqlite3.connect('flask_app/data/analysed_data.sql')
sql_cursor = sql_conn.cursor()

In [42]:
sessions = pd.read_sql('SELECT * FROM sessions', sql_conn)
sessions

,ip_address,session_ID,time,user_ID,user_agent,window_height,window_width
0,127.0.0.1,HUChCQjZu4ikoyUaD5d8XtsFl5/a5gOw,2016-08-31 12:18:18.914539,/60iePH1px+exAlkBpLnzjTYVEUPENso,Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Ma...,667,375
1,127.0.0.1,fi5Kbp+53S+bSOR8wdlYgfu1nSxS+nzh,2016-08-31 14:44:57.808794,/60iePH1px+exAlkBpLnzjTYVEUPENso,Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Ma...,667,375


In [43]:
name_lookups = pd.read_sql('SELECT * FROM name_lookups', sql_conn)
name_lookups.tail(10)

,name_1,name_2,region,session_ID,sex,time,user_ID
10,Jane,,Brussel,M9TlgWSBJFRJCcsBpVfMHrv0QVEGhqVR,F,2016-08-31 12:02:19.808883,/60iePH1px+exAlkBpLnzjTYVEUPENso
11,Jane,,Vlaanderen,M9TlgWSBJFRJCcsBpVfMHrv0QVEGhqVR,F,2016-08-31 12:02:24.720175,/60iePH1px+exAlkBpLnzjTYVEUPENso
12,Jane,,Belgie,M9TlgWSBJFRJCcsBpVfMHrv0QVEGhqVR,F,2016-08-31 12:02:29.096497,/60iePH1px+exAlkBpLnzjTYVEUPENso
13,Jane,Carolina,Belgie,M9TlgWSBJFRJCcsBpVfMHrv0QVEGhqVR,F,2016-08-31 12:02:35.910031,/60iePH1px+exAlkBpLnzjTYVEUPENso
14,Jane,Sabrina,Belgie,M9TlgWSBJFRJCcsBpVfMHrv0QVEGhqVR,F,2016-08-31 12:02:46.022073,/60iePH1px+exAlkBpLnzjTYVEUPENso
15,Karel,Pieter,Belgie,HUChCQjZu4ikoyUaD5d8XtsFl5/a5gOw,M,2016-08-31 12:18:26.017067,/60iePH1px+exAlkBpLnzjTYVEUPENso
16,Geraldine,,Belgie,fi5Kbp+53S+bSOR8wdlYgfu1nSxS+nzh,F,2016-08-31 14:45:06.823516,/60iePH1px+exAlkBpLnzjTYVEUPENso
17,Geraldine,,Wallonie,fi5Kbp+53S+bSOR8wdlYgfu1nSxS+nzh,F,2016-08-31 14:45:28.586272,/60iePH1px+exAlkBpLnzjTYVEUPENso
18,Géraldine,,Wallonie,fi5Kbp+53S+bSOR8wdlYgfu1nSxS+nzh,F,2016-08-31 14:45:39.644018,/60iePH1px+exAlkBpLnzjTYVEUPENso
19,Géraldine,,Belgie,fi5Kbp+53S+bSOR8wdlYgfu1nSxS+nzh,F,2016-08-31 14:45:44.730337,/60iePH1px+exAlkBpLnzjTYVEUPENso


In [59]:
feedback = pd.read_sql('SELECT * FROM feedback', sql_conn)
feedback.tail(10)

,feedback,name,session_ID,time,user_ID,sex
240,no_like,Clarisse,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:29:23.867005,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
241,no_like,Nina,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:29:34.153996,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
242,no_like,Leticia,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:29:46.348439,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
243,no_like,Meryem,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:29:55.222494,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
244,no_like,Soumaya,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:30:06.873446,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
245,no_like,Annabelle,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:30:15.229018,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
246,no_like,Anisoara,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:30:28.180598,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
247,no_like,Ophélie,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:30:43.590739,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
248,no_like,Mariame,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:30:53.310945,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
249,no_like,Thérèse,x6NpUZUAagSGBA6MKnReceD4YwMrBctl,2016-09-03 20:31:03.155792,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F


## Check Caroline haar feedback

In [62]:
feedback_caro = feedback.loc[feedback['user_ID']=='Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI',:]
print('Lengte van de feedback: %s ' %str(feedback_caro.shape))
print('Value counts : %s' %str(feedback_caro['feedback'].value_counts()))

Lengte van de feedback: (250, 6) 
Value counts : no_like               215
like                   31
changed_to_no_like      4
Name: feedback, dtype: int64
